<table style="border-collapse: collapse;">
  <tr>
    <td style="padding: 5px; background-color: rgba(255, 0, 0, 0.5); font-family: 'Pangolin', cursive; font-size: 60%; color: pink; text-align: center; border-radius: 20px;"><b>NAME:</b></td>
    <td style="padding: 5px;">Jeyasurya S</td>
  </tr>
  <tr>
    <td style="padding: 5px; background-color: rgba(255, 0, 0, 0.5); font-family: 'Pangolin', cursive; font-size: 60%; color: pink; text-align: center; border-radius: 20px;"><b>REG NO:</b></td>
    <td style="padding: 5px;">312821243010</td>
  </tr>
  <tr>
    <td style="padding: 5px; background-color: rgba(255, 0, 0, 0.5); font-family: 'Pangolin', cursive; font-size: 60%; color: pink; text-align: center; border-radius: 20px;"><b>EMAIL ID:</b></td>
    <td style="padding: 5px;">21aids11@act.edu.in</td>
  </tr>
</table>


In [ ]:
import numpy as np
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

: 

# <p style="background-color:rgba(255,0,0,0.5);font-family: 'Pangolin', cursive;font-size:150%;color:pink;text-align:center;border-radius:20px;"><b>Covid-19 Analys </b></p>
![](https://cdn.evrimagaci.org/Aabf8iLZ1iI8KE31fQuWkWeRUnA=/evrimagaci.org%2Fpublic%2Fcontent_media%2F93aa0b6a1659d5669e2fc32b76aef1f7.jpg)

# <p style="background-color:rgba(255,0,0,0.5);font-family: 'Pangolin', cursive;font-size:125%;color:green;text-align:center;border-radius:20px 20px;"><b>Loading the dataset</b></p>

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from folium.features import Choropleth
import folium
from folium.features import Tooltip
import seaborn as sns

: 

In [ ]:
df = pd.read_csv("/kaggle/input/covid-world-vaccination-progress/country_vaccinations_by_manufacturer.csv")

: 

In [ ]:
df.head(10)

: 

In [ ]:
df["location"].nunique()

: 

In [ ]:
df.isnull().sum()

: 

In [ ]:
df.dtypes

: 

In [ ]:
df['date'] = pd.to_datetime(df['date'])

: 

In [ ]:
data=pd.DataFrame(columns=['Country', 'Vaccine', 'Total_vaccine'])
for country in df["location"].unique():
    for vaccine in df["vaccine"].unique():
        filtered_data = df[(df['location'] == country) & (df['vaccine'] == vaccine)]
        total_count = filtered_data['total_vaccinations'].max()
        data = pd.concat([data, pd.DataFrame({'Country': [country], 'Vaccine': [vaccine], 'Total_vaccine': [total_count]})], ignore_index=True)

: 

In [ ]:
data.head(10)

: 

In [ ]:
data.dropna(axis=0,inplace=True)

: 

In [ ]:
data.head(20)

: 

# <p style="background-color:rgba(255,0,0,0.3);font-family: 'Pangolin', cursive;font-size:125%;color:black;text-align:center;border-radius:20px 20px;"><b>Most commonly used vaccines in countries</b></p>

In [ ]:
data_2=pd.DataFrame(columns=['Country', 'Vaccine'])
data["Total_vaccine"] = pd.to_numeric(data["Total_vaccine"], errors="coerce")
for country in data["Country"].unique():
    new_data = data[data["Country"] == country]
    max_vaccine = new_data.loc[new_data["Total_vaccine"].idxmax(), "Vaccine"]
    data_2 = pd.concat([data_2, pd.DataFrame({'Country': [country], 'Vaccine': [max_vaccine]})], ignore_index=True)

: 

In [ ]:
data_2.head() 

: 

In [ ]:
data_2["Vaccine"].value_counts().plot(kind="bar",
                                    color=["Red","Gray","Gray","Gray"])

: 

# <p style="background-color:rgba(255,0,0,0.3);font-family: 'Pangolin', cursive;font-size:125%;color:black;text-align:center;border-radius:20px 20px;"><b>Average daily vaccination count in countries</b></p>

In [ ]:
number_of_days = (df["date"].max() -df["date"].min() ).days

: 

In [ ]:
dtfrm=data[data["Vaccine"]=="Pfizer/BioNTech"]
dtfrm = dtfrm.drop(dtfrm[dtfrm['Country'] == 'European Union'].index)

: 

In [ ]:
dtfrm.head(10)

: 

In [ ]:
dtfrm["average_vaccination_count"] = dtfrm["Total_vaccine"] / number_of_days
dtfrm["average_vaccination_count"] =dtfrm["average_vaccination_count"].astype(int)

: 

In [ ]:
dtfrm.head(15)

: 

In [ ]:
dtfrm.set_index("Country",inplace=True)

: 

In [ ]:
color=["Lightblue","Purple","Green","Orange","darkgoldenrod","tan","Gray","Blue","Pink","Lightgreen"]
dtfrm["average_vaccination_count"].sort_values(ascending=False).head(10).plot(kind="bar",color=color)

: 

# <p style="background-color:rgba(255,0,0,0.3);font-family: 'Pangolin', cursive;font-size:125%;color:black;text-align:center;border-radius:20px 20px;"><b>Number of countries where vaccines are used</b></p>

In [ ]:
number_of_vaccines = data.groupby('Vaccine')['Country'].nunique()

: 

In [ ]:
number_of_vaccines.sort_values(ascending=False).plot(kind="bar",color="r")

: 

# <p style="background-color:rgba(255,0,0,0.3);font-family: 'Pangolin', cursive;font-size:125%;color:black;text-align:center;border-radius:20px 20px;"><b>Choropleth map of the most used vaccine</b></p>


In [ ]:
fig = px.choropleth(data_frame=dtfrm,
                    locations=dtfrm.index,
                    locationmode='country names',
                    color='Total_vaccine',
                    color_continuous_scale='YlOrRd',
                    title='Ülkelerde Yapılan Biontech Aşıları')

fig.update_layout(title_x=0.5)

: 

In [ ]:
m = folium.Map(location=[0, 0], zoom_start=2)
Choropleth(
    geo_data='https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json', 
    name='choropleth',
    data=dtfrm,  
    columns=[dtfrm.index, 'Total_vaccine'],  
    key_on='feature.properties.name', 
    fill_color='YlOrRd',  
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Aşı Sayısı', 
).add_to(m)

: 

In [ ]:
m

: 